# [Day 13](https://adventofcode.com/2024/day/13)

## Part 1

In [1]:
%pip install cvxpy

Note: you may need to restart the kernel to use updated packages.


In [91]:
import re
import cvxpy as cp
from typing import List
import numpy as np

def load_game_data(puzzel_input: str) -> List[str]:
    with open(puzzel_input, "r") as f:
        return f.read().split('\n\n')

def solve_integer_system(A:np.ndarray, b:np.ndarray, c:np.ndarray):
    """
    Solve Ax = b with integer constraints using cvxpy.
    """

    # Define integer variables
    x = cp.Variable(A.shape[1], integer=True)

    # Define the objective function (minimizing c^T x)
    objective = cp.Minimize(c @ x)

    # Constraints
    constraints = [A @ x == b]

    # Solve the problem
    prob = cp.Problem(objective, constraints)
    # Try and solve the problem
    try:
        prob.solve()
        # Check if problem was solved successfully
        if prob.status == "optimal":
            return x.value
        else:
            return None
    except cp.error.SolverError:
        print("Solver error occurred")
        return None

def parse_game(game: str):
    patterns = {
        "A": r"Button A: X\+(\d+), Y\+(\d+)",
        "B": r"Button B: X\+(\d+), Y\+(\d+)",
        "Prize": r"Prize: X=(\d+), Y=(\d+)",
    }

    # Parse button coordinates
    match_a = re.search(patterns["A"], game)
    match_b = re.search(patterns["B"], game)
    match_prize = re.search(patterns["Prize"], game)

    if not (match_a and match_b and match_prize):
        raise ValueError("Invalid game input format.")

    xa, ya = map(int, match_a.groups())
    xb, yb = map(int, match_b.groups())
    xp, yp = map(int, match_prize.groups())

    # Construct matrices
    button_matrix = np.array([[xa, xb], [ya, yb]])
    prize_vector = np.array([[xp], [yp]])
    return button_matrix, prize_vector

def fewest_tokens(game: str) -> int:
    """
    Calculate the fewest tokens needed to achieve the prize using button presses.
    """
    A, b = parse_game(game)
    # 3 tokens to push A button and 1 token to push B button.
    token_vector = np.array([3, 1])
    # Solve the system
    n_vector = solve_integer_system(A, b, token_vector)
    if n_vector is not None:
        # We don't allow more than 100 button presses per button.
        if all(n <= 100 for n in n_vector):
            return n_vector@token_vector
    return 0

In [93]:
games = load_game_data("data.txt")
total_tokens = 0
for game in games:
    total_tokens += fewest_tokens(game)
print(f"Total tokens is: {int(total_tokens)}")

Total tokens is: 38714


## Part 2

In [94]:
import re
import cvxpy as cp
from typing import List
import numpy as np

def load_game_data(puzzel_input: str) -> List[str]:
    with open(puzzel_input, "r") as f:
        return f.read().split('\n\n')

def solve_integer_system(A:np.ndarray, b:np.ndarray, c:np.ndarray):
    """
    Solve Ax = b with integer constraints using cvxpy.
    """

    # Define integer variables
    x = cp.Variable(A.shape[1], integer=True)

    # Define the objective function (minimizing c^T x)
    objective = cp.Minimize(c @ x)

    # Constraints
    constraints = [A @ x == b]

    # Define the problem
    prob = cp.Problem(objective, constraints)

    # Try and solve the problem
    try:
        prob.solve()
        # Check if problem was solved successfully
        if prob.status == 'optimal':
            return x.value
        else:
            return None
    except cp.error.SolverError:
        print("Solver error occurred")
        return None

def parse_game(game: str):
    patterns = {
        "A": r"Button A: X\+(\d+), Y\+(\d+)",
        "B": r"Button B: X\+(\d+), Y\+(\d+)",
        "Prize": r"Prize: X=(\d+), Y=(\d+)",
    }

    # Parse button coordinates
    match_a = re.search(patterns["A"], game)
    match_b = re.search(patterns["B"], game)
    match_prize = re.search(patterns["Prize"], game)

    if not (match_a and match_b and match_prize):
        raise ValueError("Invalid game input format.")

    xa, ya = map(int, match_a.groups())
    xb, yb = map(int, match_b.groups())
    xp, yp = map(int, match_prize.groups())

    # Construct matrices
    button_matrix = np.array([[xa, xb], [ya, yb]])
    # Number to add to price vector
    num = 10000000000000
    prize_vector = np.array([[xp + num], [yp + num]])
    
    return button_matrix, prize_vector

def fewest_tokens(game: str) -> int:
    """
    Calculate the fewest tokens needed to achieve the prize using button presses.
    """
    A, b = parse_game(game)
    # 3 tokens to push A button and 1 token to push B button.
    token_vector = np.array([3, 1])
    # Solve the system
    n_vector = solve_integer_system(A, b, token_vector)
    if n_vector is not None:
        return n_vector@token_vector
    return 0

In [102]:
games = load_game_data("data.txt")
total_tokens = 0
for game in games:
    total_tokens += fewest_tokens(game)
print(f"Total tokens is: {int(total_tokens)}")

Total tokens is: 52963359165012
